## 🧰 Initial Settings

### 📦 Import Standard Libraries

In [10]:
import os, glob, shutil, random

### 📥 Download Dataset from Kaggle

In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi

os.environ["KAGGLE_CONFIG_DIR"] = os.getcwd()

api = KaggleApi()
api.authenticate()

DATASET_SLUG = "hhshamimkhan/face-emotion-datasets"
api.dataset_download_files(DATASET_SLUG, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/hhshamimkhan/face-emotion-datasets


## 🛠️ Dataset Preparation for YOLO

### ⚙️ Dataset Paths, Split Ratios, and Configuration Settings

In [3]:
SRC_ROOT = "Face emotion datasets"
OUT_ROOT = "dataset_yolo"
R_TRAIN, R_VAL = 0.70, 0.15
SEED = 42
VALID_EXT = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

random.seed(SEED)

### 🔎 Discover Classes and Assign IDs

In [4]:
# 1) Descubrir clases (orden alfabético estable)
classes = sorted(
    [d for d in os.listdir(SRC_ROOT) if os.path.isdir(os.path.join(SRC_ROOT, d))]
)
name2id = {c: i for i, c in enumerate(classes)}
print("Clases -> IDs:")
name2id

Clases -> IDs:


{'Anger': 0,
 'Content': 1,
 'Disgust': 2,
 'Fear': 3,
 'Happy': 4,
 'Neutral': 5,
 'Sad': 6,
 'SillyFace': 7,
 'Surprise': 8}

### 🗂️ Split Dataset into Train/Val/Test and Generate YOLO Labels

In [5]:
for s in ["train", "val", "test"]:
    os.makedirs(os.path.join(OUT_ROOT, "images", s), exist_ok=True)
    os.makedirs(os.path.join(OUT_ROOT, "labels", s), exist_ok=True)

for cls in classes:
    imgs = [
        p
        for p in glob.glob(os.path.join(SRC_ROOT, cls, "*"))
        if p.lower().endswith(VALID_EXT)
    ]

    random.shuffle(imgs)

    n = len(imgs)
    n_tr = int(n * R_TRAIN)
    n_va = int(n * R_VAL)
    n_te = n - n_tr - n_va

    splits = [
        ("train", imgs[:n_tr]),
        ("val", imgs[n_tr : n_tr + n_va]),
        ("test", imgs[n_tr + n_va :]),
    ]

    for split, items in splits:
        for src_img in items:

            fn = os.path.basename(src_img)
            shutil.copy2(src_img, os.path.join(OUT_ROOT, "images", split, fn))

            with open(
                os.path.join(
                    OUT_ROOT, "labels", split, os.path.splitext(fn)[0] + ".txt"
                ),
                "w",
            ) as f:
                f.write(f"{name2id[cls]} 0.5 0.5 1.0 1.0\n")

### 📄 Create YOLO Configuration File (data.yaml)

In [6]:
with open(os.path.join(OUT_ROOT, "data.yaml"), "w", encoding="utf-8") as f:
    f.write(
        f"path: {OUT_ROOT}\ntrain: images/train\nval: images/val\n"
        f"test: images/test\nnames: [{', '.join(classes)}]\n"
    )

In [7]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")